python中有3个包可以做这个事情  
* hdfs
* hdfs3
* pyhdfs

公司hdfs环境 http://10.23.50.178:50070， 需要连Uclound vpn

# hdfs
https://hdfscli.readthedocs.io/en/latest/  
https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings


In [9]:
#!pip install hdfs
!pip freeze | grep hdfs

hdfs==2.5.8
hdfs3==0.3.1


## Instantiating a client

In [1]:
from hdfs import InsecureClient
# 50070: NameNode web管理端口
client = InsecureClient('http://10.23.50.178:50070', user='hdfs')
# from hdfs.client import Client
# client = Client("http://10.23.50.178:50070")  

## Exploring the file system

In [4]:
# Retrieving a file or folder content summary.
content = client.content('/tmp')  # 如果是相对路径，会被默认到/user/hdfs/+相对路径
content

{'directoryCount': 49,
 'fileCount': 35,
 'length': 80829331,
 'quota': -1,
 'spaceConsumed': 215585123,
 'spaceQuota': -1,
 'typeQuota': {}}

In [5]:
# Listing all files inside a directory.
client.list("/tmp")

['druid-indexing', 'entity-file-history', 'hive']

In [6]:
client.status('/py.txt')

{'accessTime': 1566203383342,
 'blockSize': 134217728,
 'childrenNum': 0,
 'fileId': 4694548,
 'group': 'hdfs',
 'length': 11,
 'modificationTime': 1566203383359,
 'owner': 'hdfs',
 'pathSuffix': '',
 'permission': '644',
 'replication': 1,
 'storagePolicy': 0,
 'type': 'FILE'}

In [7]:
# Renaming ("moving") a file.
client.rename('/py.txt', '/tmp/pyt.txt')

In [8]:
# Deleting a file or folder.
client.delete('/tmp/pyt.txt', recursive=True)

True

In [9]:
# Download a file or folder locally.
client.download('/aml', 'aml', n_threads=4)

'/Users/luoyonggui/PycharmProjects/mayiexamples/db/aml'

## Reading files

In [3]:
# Loading a file in memory.
with client.read('features') as reader:
  features = reader.read()

# Directly deserializing a JSON object.
with client.read('model.json', encoding='utf-8') as reader:
  from json import load
  model = load(reader)

b'hello world'


If a chunk_size argument is passed, the method will return a generator instead, making it sometimes simpler to stream the file’s contents.

In [ ]:
# Stream a file.
with client.read('features', chunk_size=8096) as reader:
  for chunk in reader:
    pass

Similarly, if a delimiter argument is passed, the method will return a generator of the delimited chunks.

In [ ]:
with client.read('samples.csv', encoding='utf-8', delimiter='\n') as reader:
  for line in reader:
    pass

### 读取parquet文件，存入本地

In [4]:
with client.read("/aml/b.parquet") as reader: 
    data = reader.read()
    with open('bb.parquet', mode='wb') as f:
        f.write(data)
#     ParquetFile()
#     df = pd.read_parquet(reader.read())

In [5]:
df = pd.read_parquet('bb.parquet')

In [3]:
import pandas as pd

In [6]:
df.head()

,feeDetailNo,recipeDate,applyPayLevel,chargeType,doseForm,medicalSpecification,insuranceType,projectType,registerNo,hospNo,...,cd_overallPay,rs_overallPay,accountPay,rescuePay,amount,selfAmount,quantity,money,applyAmount,cdpAmount
0,18eb4cdea7d393a2,2017-05-11 00:00:00,甲,治疗费,无,0,N,诊疗费,000b7c75ef6ca29d,330599100005,...,0.0,23.10,0.0,0.0,0.0,0.00,4.0,68.00,68.0,0.0
1,f54b5e83c95f4099,2017-05-11 00:00:00,乙,西药费,片 剂,0.1g*24片,N,药品费,000b7c75ef6ca29d,330599100005,...,0.0,23.10,0.0,0.0,0.0,3.22,2.0,61.22,58.0,0.0
2,e8cad9b577685f66,2017-05-11 00:00:00,乙,西药费,片 剂,10mg*7片,N,药品费,000b7c75ef6ca29d,330599100005,...,0.0,23.10,0.0,0.0,0.0,5.12,4.0,95.12,90.0,0.0
3,b6f5d28eff70359e,2017-05-11 00:00:00,乙,西药费,其它,0.15g/15g*1支,N,药品费,000b7c75ef6ca29d,330599100005,...,0.0,23.10,0.0,0.0,0.0,1.00,2.0,20.00,19.0,0.0
4,57b9100b2a2408eb,2017-03-06 00:00:00,乙,中成药费,无,0.3g*16s,N,药品费,0010b8084da5faf7,330523100001,...,0.0,70.95,0.0,0.0,0.0,7.36,4.0,149.36,142.0,0.0


## Writing files

In [ ]:
# Writing part of a file.
with open('samples') as reader, client.write('samples') as writer:
  for line in reader:
    if line.startswith('-'):
      writer.write(line)

# Writing a serialized JSON object.
with client.write('model.json', encoding='utf-8') as writer:
  from json import dump
  dump(model, writer)

For convenience, it is also possible to pass an iterable data argument directly to the method.

In [ ]:
model = {}

In [ ]:
# This is equivalent to the JSON example above.
from json import dumps
client.write('model.json', dumps(model))

# pyhdfs

In [7]:
from pyhdfs import HdfsClient

In [2]:
client=HdfsClient(hosts='10.23.50.178:50070',user_name='hdfs')#只有hadoop用户拥有写权限

In [3]:
str='hello world'
client.create('/py.txt',str)#创建新文件并写入字符串

# hdfs3

In [11]:
from hdfs3 import HDFileSystem
from fastparquet import ParquetFile

In [12]:
hdfs = HDFileSystem(host='10.23.50.178', port=8020, user='hdfs')

In [5]:
hdfs.ls('/')

['//aml',
 '//app-logs',
 '//apps',
 '//ats',
 '//atsv2',
 '//data',
 '//hdp',
 '//livy2-recovery',
 '//mapred',
 '//mr-history',
 '//ranger',
 '//spark2-history',
 '//tmp',
 '//user',
 '//warehouse']

In [6]:
hdfs.ls('/aml')

['/aml/a.parquet',
 '/aml/a1.parquet',
 '/aml/a2.parquet',
 '/aml/b.parquet',
 '/aml/hivemetastore.log']

In [7]:
import pandas as pd

In [ ]:
with hdfs.open('/aml/b.parquet') as f:
    df = pd.read_parquet(f)

In [4]:
sc = hdfs.open

In [ ]:
pf = ParquetFile('/aml/b.parquet', open_with=sc)